## Cropbox

[`Cropbox`](https://github.com/tomyun/Cropbox.jl) is a modeling framework witten in Julia that was developed to support and facilitate plant and crop modeling activities: building, applying, and learning about plant and crop models [@Yun2023]. See [Cropbox documentation](https://cropbox.github.io/Cropbox.jl/stable/) for additional tutorials and information.

### Model Specification

We now start using Cropbox framework to make a very simple exponential growth model adapted from an example introduced in the manual of [Fortran Simulation Translator](https://library.wur.nl/WebQuery/wurpubs/302463) which used to be an underlying framework for many early plant and crop growth models.

In [ ]:
using Cropbox

Here is an equation of the exponential growth model.

$$\frac{\Delta x}{\Delta t} = ax$$

This mathematical expression basically states that variable `x` changes over time at the rate of `a` relative to the existing value of `x`. If `a` is 0.1 then `x` grows 10% over the time interval of $\Delta t$. We now represent this model in a **system** named `S` which contains the state variable `x` and other supporting variables. We give the initial value of $x$ as 1.

In [ ]:
@system S(Controller) begin
    i       => 1   ~ preserve
    a       => 0.1 ~ preserve(parameter)
    r(a, x) => a*x ~ track
    x(r)           ~ accumulate(init = i)
end

Here we declared four variables. - `i`: variable containing initial value of `x` which never changes (*preserved*) - `a`: variable containing constant **parameter** of exponential growth - `r`: rate variable which needs to be calculated or *tracked* every time step. This is the rate change of `x` over time. - `x`: state variable which *accumulates* by rate `r` over time with initial value `i`

In `Cropbox`, we can visualize dependency graph between variables defined in our system `S` by using some internal functions provided by the framework.

In [ ]:
Cropbox.dependency(S)

For example, it's clear to see `r` depends on two variables `x` and `a`. There are some auxiliary nodes (prefixed with $\circ$/$\star$) and implicit variables (*i.e.* `config`, `context`) not declared in our model specification and we will discuss them later.

### Model Simulation

Once we have our model repressented in a system `S`, we can run the model for simulation. While you can directly instantiate your model by calling `instance(S)`, here we will use a helper function `simulate()` for simplicity. It will internally instantiate the model, update it for a given number of iterations and return the result packed in a `DataFrame`. By default, columns will include `tick` variable from internal `Clock` and all state variables defined in the target system.

In [ ]:
r = simulate(S; stop = 10)

Note that `tick` is in hourly units (`hr`) by default.

#### Plotting

As the result is backed by [DataFrames.jl](https://github.com/JuliaData/DataFrames.jl), we can easily hand it off to other Julia packages for further analysis. Cropbox framework also provides a handy `visualize()` function for simple visualization.

In [ ]:
visualize(r, :time, :x; kind = :line)

#### Configuration

Note that the rate of growth `a` is a *parameter* in our model. A value of `0.1` was paired with variable `a` previously. In modeling, we often need to change the value of a parameter for different systems or species. We can change its value before running the model by assigning a different value. Let's try smaller value of 0.05 instead of default 0.1. We will name this new parameter configuration `c` which will pair the system `S` with `a` variable with a value of 0.05.

In [ ]:
c = S => :a => 0.05

In [ ]:
@config(c)

The initial values of variables (mostly `preserve` kind in practice) tagged as `parameter` can be set in a dictionary-like structure supplied to `simulate()` or `instance()` by keyword argument named `config`.

In [ ]:
r2 = simulate(S, config = c, stop = 10);

In [ ]:
visualize(r2, :time, :x; kind = :line)

Now we can confirm that the exponential growth became much slower than the previous simulation by using a smaller rate parameter.

### Model Internals

When we specify our model with `@system` macro, Cropbox automatically generates some code behind the scene for handling model setup and update. With our exponential growth model, it would mean we get `struct` named `S` and `update!(::S, ..)` for example. We can take a look at the actual code generatd during this process by using [`@macroexpand`](https://docs.julialang.org/en/v1/base/base/#Base.macroexpand).

In [ ]:
@macroexpand @system S(Controller) begin
    i => 1         ~ preserve
    a => 0.1       ~ preserve(parameter)
    r(a, x) => a*x ~ track
    x(r)           ~ accumulate(init = i)
end;

#### **Syntax**

`@system` macro accepts lines of variable declaration specified by its own syntax. They are loosely based on Julia syntax sharing common expressions and operators, but have distinct semantics explained as below. In other words, they form a domain-specific language (DSL) specifically designed for our crop modeling framework.

`name[(args..; kwargs..)][: alias] [=> body] ~ [state][::type][(tags..)]`

-   `name`: variable name (usually short abbreviation)
-   `args`: automatically bound depending variables
-   `kwargs`: custom bound depending variables (only for *call* now)
-   `alias`: alternative name (long description)
-   `body`: code snippet (state/type specific, `begin .. end` block for multiple lines)
-   `state`: verb indicating kind of state (empty if not `State`-based)
-   `type`: internal type (*i.e.* `Float64` by default for most `State` variable)
-   `tags`: variable specific options (*i.e.* unit, min/max, etc.)

Here is a hypothetical model specification showing various uses of syntax elements.

In [ ]:
@system MySystem begin
    a: my_precious_param => 1 ~ preserve::Int(parameter)
    b(a) => begin
        2a
    end ~ track
    c                     ~ ::String(override)
    f(b; x, y) => b*(x+y) ~ call
    g(f)       => f(1, 2) ~ track
end

By the way, although looking verbose and suspicious, `MySystem` is still on a valid specification and you can visualizae its dependency graph of state variables without any problem.

In [ ]:
Cropbox.dependency(MySystem)

#### State

There are a number of built-in `State` verbs that can be used in model specification.

-   `hold`: marks a placeholder for variable shared between mixins.
-   `wrap`: passes a state variable to other fucnction as is with no unwrapping its value.
-   `advance`: manages a time-keeping variable; `time` and `tick` from `Clock`.
-   `preserve`: keeps initially assigned value with no further updates; constants, parameters.
-   `tabulate`: makes a two dimensional table with named keys; *i.e.* partitioning table.
-   `interpolate`: makes a curve function interpolated with discrete values; *i.e.* soil characteristic curve.
-   `track`: evaluates variable expression as is for each update.
-   `remember`: keeps tracking variable until a certain condition is met; essentially `track` turning into `preserve`.
-   `provide`: manages a table of time-series in DataFrame.
-   `drive`: fetches the current value from a time-series; maybe supplied by `provide`.
-   `call`: defines a partial function bound with some variables.
-   `integrate`: calculates integral using Gaussian method; not for time domain.
-   `accumulate`: emulates integration of rate variable over time; essentially Euler method.
-   `capture`: calculates difference between integration for each time step.
-   `flag`: sets a boolean flag; essentially `track::Bool`.
-   `produce`: attaches a new instance of system dynamically constructed; *i.e.* root structure growth.
-   `bisect`: solves nonlinear equation using bisection method; *i.e.* gas-exchange model coupling.
-   `solve`: solves polynomial equation symbolically; *i.e.* quadratic equations in photosynthesis model.

Internally, each kind directly maps to a Julia `struct` with its name capitalized. For example, `track` kind maps to `Track` struct.

#### System

`System` is a container that contains a number of *variables*. Once defined, its structure gets fixed that no additional variables can get added or removed, but the state of variables can be still updated throughout time steps.

`System` may include other `System` as its component. In such case, kind of `state` would be empty and type of included `System` should be defined in the variable declaration. Indeed, every `System` by default includes a `System` called `Context` referred by `context` variable. We can visualize hierarchy of systems using `Cropbox.hierarchy()`.

For example, we can visually confirm that `MySystem` *has* a `Context` which *has* a `Clock`.

In [ ]:
Cropbox.hierarchy(MySystem)

In order to get a model that can be run, we need to make an instance of `System`. It can be done by calling `instance()` function on the type of `System`.

In [ ]:
s = instance(S)

Here, we just made an instance of `S` which is referred by a variable name `s`.

#### Simulation

Now we can call `simulate!()` to update its internal states for one time step.

In [ ]:
simulate!(s)

If you want to run for more than one time step, the number of iterations can be specfied.

In [ ]:
simulate!(s; stop = 2)

Instead of calling `simulate!()` after `instance()`, `simulate()` can do both at the same time. Note that `simulate()` works with the type of `System` whereas `simulate!()` works with an instance of such `System`. Note that Julia follows a convention that function ending with `!` can change or mutate any arguments supplied to the function.

In [ ]:
simulate(S; stop = 3)

#### Inspection

Cropbox provides `look()` function (and equivalent `@look` macro) which is a handy tool for inspecting internal states of `System` and/or instance of the system. Cropbox also provide `dive()` function for interactive navigating in and out of the variables, but it only works in terminal REPL environment, not in Jupyter Notebook [yet](https://github.com/JuliaLang/IJulia.jl/issues/801).

In [ ]:
look(s)

In [ ]:
@look s.a

#### Controller

At this point, you may want to run this model as we did before with `simulate()`, but it would turn out our `MySystem` can't be run as a standalone model. Due to technical reasons for performance, any `System` that wants to be a direct target of instantiation needs to to include `Controller` mixin in its specification. Remember we had `@system S(Controller) ..`, not just `@system S ..` in our first example.

#### Mixin

[Mixin](https://en.wikipedia.org/wiki/Mixin) is a piece of `System` specification that can be included in the specification of larger `System`. Each mixin implements an own set of variables and some of them may be linked with variables from other `System` when a new `System` is formed. Mixins are declared by giving a list of them enclosed within parentheses after the name of `System` in the specification.

Here is an example of mixin where `S3` is composed by including `S1` and `S2` as mixins.

In [ ]:
@system S1 begin
    a    => 1  ~ preserve
    b(a) => 2a ~ track
end

In [ ]:
@system S2 begin
    a       => 2   ~ preserve
    b(a, c) => a*c ~ track
    c       => 1   ~ preserve
end

In [ ]:
@system S3(S1, S2) begin
    d(a) => 3a ~ preserve
end

We can take a look at the actual specification of `S3` after mixins were included by calling `Cropbox.source()`.

In [ ]:
Cropbox.source(S3)

Note that `a` from `S1` with default value of 1 was overridden by `S2` where default value was 2. The same went for variable `b`. In other words, order of mixins are important.

It may help understanding mixins by visualizaing their structure.

In [ ]:
Cropbox.dependency(S1)

In [ ]:
Cropbox.dependency(S2)

In [ ]:
Cropbox.dependency(S3)

In [ ]:
Cropbox.hierarchy(S3)

Here `S3` *includes* `S1` and `S2`. `S3` *has* a `Context` which has a `Clock`.

### Notes

1.  Some functions (*i.e.* `simulate()`, `instance()`) used in this notebook were simply called without module name (*i.e.* `Cropbox.`), while some had to be called with a prefix (*e.g.* `Cropbox.hierarchy()`, `Cropbox.source()`). By default, the latter group of functions are not exported to the user scope, because they were mostly designed for internal use and their forms and behaviors are likely to change in the future. So please be mindful when you need to use them.

2.  This tutorial assumed readers have a basic understanding of Julia language. For more information, please read the official [Julia documentation](https://docs.julialang.org/).